# Assigment 3

Tuğçe Kul

### Importing Required modules and libraries

In [30]:
import pandas as pd
import numpy as np
from pulp import *

### Defining Required Parameters

In [31]:
# the seed number should be your student ID.
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(21069039)

# Requirment of the production (proudction hour)
RandomFtoD1 = np.round(np.random.uniform(low=1, high=10, size=4), 0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (2, 2))
BesiktasHour = pd.DataFrame(
    RandomFtoD2, columns=["T-shirts", "Jeans"], index=["Cutting", "Sewing"]
)
BesiktasHour.columns = pd.MultiIndex.from_product([["Beşiktaş"], BesiktasHour.columns])

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low=1, high=10, size=4), 0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (2, 2))
DavutpasaHour = pd.DataFrame(
    RandomDtoS2, columns=["T-shirts", "Jeans"], index=["Cutting", "Sewing"]
)
DavutpasaHour.columns = pd.MultiIndex.from_product(
    [["Davutpaşa"], DavutpasaHour.columns]
)

# Your Required Information About Production Hour
randomDataTable = pd.concat([BesiktasHour, DavutpasaHour], axis=1)
print(randomDataTable)

        Beşiktaş       Davutpaşa      
        T-shirts Jeans  T-shirts Jeans
Cutting        9     1         1     2
Sewing         2     9        10     9


In [32]:
Products = ["T-Shirts", "Jeans"]

Centers = ["Besiktas", "Davutpasa"]

Procesess = ["Cutting", "Sewing"]

Profit = {"T-Shirts": 12, "Jeans": 16}


Capacity = {
    "Besiktas": {"Cutting": 180, "Sewing": 160},
    "Davutpasa": {"Cutting": 70, "Sewing": 80},
}

besiktas_tshirts_cutting = randomDataTable.iloc[0, 0]
besiktas_tshirts_sewing = randomDataTable.iloc[1, 0]
besiktas_jeans_cutting = randomDataTable.iloc[0, 1]
besiktas_jeans_sewing = randomDataTable.iloc[1, 1]
davutpasa_tshirts_cutting = randomDataTable.iloc[0, 2]
davutpasa_tshirts_sewing = randomDataTable.iloc[1, 2]
davutpasa_jeans_cutting = randomDataTable.iloc[0, 3]
davutpasa_jeans_sewing = randomDataTable.iloc[1, 3]

TimeSpent = {
    "Besiktas": [
        {"T-Shirts": {"Cutting": besiktas_tshirts_cutting, "Sewing": besiktas_tshirts_sewing}},
        {"Jeans": {"Cutting": besiktas_jeans_cutting, "Sewing": besiktas_jeans_sewing}},
    ],
    "Davutpasa": [
        {"T-Shirts": {"Cutting": davutpasa_tshirts_cutting, "Sewing": davutpasa_tshirts_sewing}},
        {"Jeans": {"Cutting": davutpasa_jeans_cutting, "Sewing": davutpasa_jeans_sewing}},
    ],
}

RawMaterial = {"T-Shirts": 4, "Jeans": 4}

### Creating The "prob" Variable To Contain The Problem Data

In [33]:
prob = LpProblem("Clothes_Production", LpMaximize)

### Defining Decision Variable

In [34]:
X = LpVariable.dicts("X", (Products, Centers), lowBound = 0, upBound = None, cat = const.LpInteger)
display(X)

{'T-Shirts': {'Besiktas': X_T_Shirts_Besiktas,
  'Davutpasa': X_T_Shirts_Davutpasa},
 'Jeans': {'Besiktas': X_Jeans_Besiktas, 'Davutpasa': X_Jeans_Davutpasa}}

### Defining Objective Function

In [35]:
prob += lpSum(X[i][x]*Profit[i] for i in Products for x in Centers)

### Defining Constraints

In [36]:
for x in Centers:
    for s in Procesess:
        prob += lpSum(item[t][s]*X[t][x] for item in TimeSpent[x] for t in item.keys() ) <= Capacity[x][s]

In [37]:
prob += lpSum(X[i][x]*RawMaterial[i] for i in Products for x in Centers) <= 120

### Solving Problem

In [38]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/emirhan/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c492c8b392b7410caf43112eedc1c6f2-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/c492c8b392b7410caf43112eedc1c6f2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 35 RHS
At line 41 BOUNDS
At line 46 ENDATA
Problem MODEL has 5 rows, 4 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 462.857 - 0.00 seconds
Cgl0004I processed model has 4 rows, 4 columns (4 integer (0 of which binary)) and 10 elements
Cutoff increment increased from 1e-05 to 3.9999
Cbc0012I Integer solution of -432 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0038I Full problem 4 rows 4 columns, reduced to 3 rows 2 columns
Cbc0012I Integer solution of -456 fou

1

### Printing Problem Status

In [39]:
print("The Problem Is:", LpStatus[prob.status])

The Problem Is: Optimal


### d.a Production Quantities: i. Jeans in Beşiktaş and in Davutpaşa, ii. T-shirts in Beşiktaş and in Davutpaşa


In [40]:
print("Jeans in Beşiktaş and Davutpaşa (Production Quantities):\n")
for v in prob.variables():
    print(v.name, "=", v.varValue)

Jeans in Beşiktaş and Davutpaşa (Production Quantities):

X_Jeans_Besiktas = 16.0
X_Jeans_Davutpasa = 8.0
X_T_Shirts_Besiktas = 6.0
X_T_Shirts_Davutpasa = 0.0


#### d.b Total amount used Cutting in Beşiktaş and in Davutpaşa, Sewing in Beşiktaş and in Davutpaşa, Cloths in Beşiktaş and in Davutpaşa


In [41]:
print("Cutting in Beşiktaş and in Davutpaşa,\nSewing in Beşiktaş and in Davutpaşa,\nCloths in Beşiktaş and in Davutpaşa")

print("Production Quantities:\n")
for x in Centers:
    for s in Procesess:
        for i in Products:
            print(f"{s} {i} in {x}: {X[i][x].varValue}")

Cutting in Beşiktaş and in Davutpaşa,
Sewing in Beşiktaş and in Davutpaşa,
Cloths in Beşiktaş and in Davutpaşa
Production Quantities:

Cutting T-Shirts in Besiktas: 6.0
Cutting Jeans in Besiktas: 16.0
Sewing T-Shirts in Besiktas: 6.0
Sewing Jeans in Besiktas: 16.0
Cutting T-Shirts in Davutpasa: 0.0
Cutting Jeans in Davutpasa: 8.0
Sewing T-Shirts in Davutpasa: 0.0
Sewing Jeans in Davutpasa: 8.0


### d.c Total profit. 


In [42]:
print("The objective Function Value (Total Profit) is: ", value(prob.objective))

The objective Function Value (Total Profit) is:  456.0


In [43]:
print("Decision Variables")
print(prob.variables())
print(prob)

Decision Variables
[X_Jeans_Besiktas, X_Jeans_Davutpasa, X_T_Shirts_Besiktas, X_T_Shirts_Davutpasa]
Clothes_Production:
MAXIMIZE
16*X_Jeans_Besiktas + 16*X_Jeans_Davutpasa + 12*X_T_Shirts_Besiktas + 12*X_T_Shirts_Davutpasa + 0
SUBJECT TO
_C1: X_Jeans_Besiktas + 9 X_T_Shirts_Besiktas <= 180

_C2: 9 X_Jeans_Besiktas + 2 X_T_Shirts_Besiktas <= 160

_C3: 2 X_Jeans_Davutpasa + X_T_Shirts_Davutpasa <= 70

_C4: 9 X_Jeans_Davutpasa + 10 X_T_Shirts_Davutpasa <= 80

_C5: 4 X_Jeans_Besiktas + 4 X_Jeans_Davutpasa + 4 X_T_Shirts_Besiktas
 + 4 X_T_Shirts_Davutpasa <= 120

VARIABLES
0 <= X_Jeans_Besiktas Integer
0 <= X_Jeans_Davutpasa Integer
0 <= X_T_Shirts_Besiktas Integer
0 <= X_T_Shirts_Davutpasa Integer

